In [2]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [28]:
dataframe = pandas.read_table("housing.data", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [42]:

print(X.shape)
print(X[0][1])
print(Y[1])

(506, 13)
18.0
21.6


In [47]:
Xd = X[1:,:]
Yd = Y[2:,]
print(Xd[0][0])
print(Yd[0])

0.02731
34.7


In [67]:
import quandl
data = quandl.get('BCHARTS/KRAKENUSD', returns='pandas')
print(data.shape)
print(dataframe.shape)

(1403, 7)
(506, 14)


In [93]:
data.tail()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2017-11-05,7366.5,7575.0,7297.0,7380.0,2604.588509,1.940652e+07,7450.897130
2017-11-06,7374.7,7449.0,6920.0,6969.0,4961.192013,3.578951e+07,7213.892964
2017-11-07,6970.3,7273.1,6942.0,7128.1,2974.537723,2.116621e+07,7115.797600
2017-11-08,7128.1,7788.0,7097.0,7463.5,6190.612581,4.588957e+07,7412.766709
2017-11-09,7460.0,7462.3,7460.0,7460.1,1.352533,1.009076e+04,7460.640985


In [77]:
btcdata = data
X = btcdata

In [97]:
import numpy as np
btcdata = data.values
# split into input (X) and output (Y) variables
X = btcdata[:,0:6]
Y = btcdata[:,6]
#btcdata.head()
#btcdata.shape
Xt = btcdata[:,0:6]
Yt = btcdata[1:,6]
np.delete((Xt[-1]))
print(Xt[-1])
print(Yt[-1])

TypeError: delete() missing 1 required positional argument: 'obj'

In [17]:
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=wider_model, nb_epoch=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))